# Azure AI Search с интеграцией NVIDIA NIM и LlamaIndex

В этом ноутбуке мы покажем, как использовать модели искусственного интеллекта NVIDIA и LlamaIndex для создания мощного конвейера Retrieval-Augmented Generation (RAG). Мы будем использовать большие языковые модели (LLM) и эмбеддинги NVIDIA, интегрировать их с Azure AI Search в качестве хранилища векторов и выполнять RAG для повышения качества и эффективности поиска.

## Преимущества
- **Масштабируемость**: Используйте крупные языковые модели NVIDIA и Azure AI Search для масштабируемого и эффективного поиска.
- **Экономичность**: Оптимизируйте поиск и извлечение данных с помощью эффективного хранения векторов и гибридных методов поиска.
- **Высокая производительность**: Сочетайте мощные LLM с векторным поиском для более быстрых и точных ответов.
- **Качество**: Обеспечьте высокое качество поиска, подкрепляя ответы LLM релевантными извлеченными документами.

## Предварительные требования
- 🐍 Python 3.9 или выше
- 🔗 [Служба Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 Ключ API NVIDIA для доступа к LLM и эмбеддингам NVIDIA через микросервисы NVIDIA NIM

## Охватываемые функции
- ✅ Интеграция LLM NVIDIA (мы будем использовать [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Эмбеддинги NVIDIA (мы будем использовать [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Расширенные режимы извлечения Azure AI Search
- ✅ Индексация документов с помощью LlamaIndex
- ✅ RAG с использованием Azure AI Search и LlamaIndex в сочетании с LLM NVIDIA

Давайте начнем!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Установка и требования
Создайте окружение Python, используя версию Python >3.10.

## Начало работы!


Чтобы начать, вам понадобится `NVIDIA_API_KEY` для использования моделей NVIDIA AI Foundation:
1) Создайте бесплатный аккаунт на [NVIDIA](https://build.nvidia.com/explore/discover).
2) Выберите модель, которая вас интересует.
3) В разделе Input выберите вкладку Python, нажмите **Get API Key**, а затем **Generate Key**.
4) Скопируйте и сохраните сгенерированный ключ как NVIDIA_API_KEY. После этого у вас будет доступ к конечным точкам.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Пример RAG с использованием LLM и встраивания
### 1) Инициализация LLM
`llama-index-llms-nvidia`, также известный как коннектор LLM от NVIDIA, позволяет подключаться к совместимым моделям и генерировать результаты с использованием моделей, доступных в каталоге API NVIDIA. Список моделей для завершения чата можно найти здесь: https://build.nvidia.com/search?term=Text-to-Text

Здесь мы будем использовать **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Инициализация встраивания  
`llama-index-embeddings-nvidia`, также известный как коннектор Embeddings от NVIDIA, позволяет подключаться к совместимым моделям и генерировать данные с их помощью, доступным в каталоге API NVIDIA. Мы выбрали модель `nvidia/nv-embedqa-e5-v5` в качестве модели для встраивания. Список моделей для текстового встраивания можно найти здесь: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Создание поискового механизма для вопросов по вашим данным

Вот пример запроса, использующего чистый поиск по векторам в Azure AI Search и привязку ответа к нашей модели LLM (Phi-3.5-MOE).


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Вот запрос с использованием гибридного поиска в Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Анализ поиска по векторам
Ответ LLM точно отражает ключевые экономические последствия, упомянутые в исходном тексте, касающиеся российского фондового рынка. В частности, указано, что российский фондовый рынок значительно упал, потеряв 40% своей стоимости, а торговля была приостановлена из-за текущей ситуации. Этот ответ хорошо соответствует предоставленной информации, показывая, что LLM правильно идентифицировал и суммировал важные детали, связанные с влиянием на фондовый рынок в результате действий России и наложенных санкций.

#### Комментарий к исходным узлам
Исходные узлы предоставляют подробное описание экономических последствий, с которыми столкнулась Россия из-за международных санкций. В тексте подчеркивается, что российский фондовый рынок потерял 40% своей стоимости, а торговля была приостановлена. Кроме того, упоминаются другие экономические последствия, такие как обесценивание рубля и более широкая изоляция российской экономики. Ответ LLM эффективно выделил ключевые моменты из этих узлов, сосредоточившись на влиянии на фондовый рынок, как было запрошено в запросе.


Теперь давайте рассмотрим запрос, в котором гибридный поиск не дает обоснованного ответа:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Гибридный поиск: анализ ответа LLM
Ответ LLM в примере гибридного поиска указывает на то, что предоставленный контекст не содержит точной даты вторжения России в Украину. Этот ответ предполагает, что LLM использует информацию из исходных документов, но признает отсутствие конкретных деталей в тексте.

Ответ точно определяет, что в контексте упоминаются события, связанные с агрессией России, но не указывается конкретная дата вторжения. Это демонстрирует способность LLM понимать предоставленную информацию, одновременно отмечая пробелы в содержании. LLM эффективно побуждает пользователя обратиться к внешним источникам или историческим записям для уточнения даты, проявляя осторожность при недостатке информации.

### Анализ исходных узлов
Исходные узлы в примере гибридного поиска содержат выдержки из речи, обсуждающей реакцию США на действия России в Украине. Эти узлы подчеркивают более широкий геополитический эффект и шаги, предпринятые США и их союзниками в ответ на вторжение, но не упоминают конкретную дату вторжения. Это соответствует ответу LLM, который правильно определяет, что в контексте отсутствует информация о точной дате.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Гибрид с повторным ранжированием: анализ ответа LLM
В примере с гибридом и повторным ранжированием ответ LLM предоставляет дополнительный контекст, отмечая, что событие произошло за шесть дней до выступления. Это указывает на то, что LLM способен сделать вывод о дате вторжения, основываясь на времени выступления, хотя для точности всё ещё необходимо знать точную дату самого выступления.

Этот ответ демонстрирует улучшенную способность использовать контекстные подсказки для предоставления более информативного ответа. Он подчёркивает преимущество повторного ранжирования, при котором LLM может получить доступ к более релевантной информации и приоритизировать её, чтобы дать более точное приближение к искомой детали (например, дате вторжения).

### Анализ исходных узлов
Исходные узлы в этом примере включают ссылки на время вторжения России, конкретно упоминая, что оно произошло за шесть дней до выступления. Хотя точная дата всё ещё не указана явно, узлы предоставляют временной контекст, который позволяет LLM дать более тонкий ответ. Включение этой детали демонстрирует, как повторное ранжирование может улучшить способность LLM извлекать и делать выводы из предоставленного контекста, что приводит к более точному и информативному ответу.


**Примечание:**  
В этом блокноте мы использовали микросервисы NVIDIA NIM из каталога API NVIDIA.  
Указанные выше API, `NVIDIA (llms)`, `NVIDIAEmbedding` и [Azure AI Search Semantic Hybrid Retrieval (встроенная повторная ранжировка)](https://learn.microsoft.com/azure/search/semantic-search-overview). Обратите внимание, что указанные выше API также поддерживают микросервисы с самостоятельным хостингом.  

**Пример:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
